## Video Planning - Chain of Judge

In diesem Projekt greifen wir auf ein neues Konzept zurück, um die Qualität und Effizienz von Lösungen zu verbessern. Wir nutzen zwei Sprachmodelle, einen Generator und einen Critic, um eine Feedback-Schleife zu bilden. Diese Methode basiert auf der sogenannten Self-Consistency mit Feedback-Schleife oder Critic-and-Generator-Framework, die es ermöglicht, sowohl die Qualität der Lösung als auch ihre Erklärbarkeit zu erhöhen.


Der Generator-Modell erstellt eine Lösung und gibt dabei durch Chain-of-Thought Prompting eine schrittweise Erklärung ab. Der Critic-Modell bewertet den Vorschlag des Generators und identifiziert mögliche Schwächen oder Fehler. Dieser Prozess wiederholt sich in einer Feedback-Schleife, wodurch der Generator seine Lösung kontinuierlich verbessern kann.


Diese innovative Methode bietet mehrere Vorteile: Sie ermöglicht eine Qualitätskontrolle durch den Critic, schrittweise Optimierung des Generators und erhöhte Erklärbarkeit durch Chain-of-Thought-Prompting. Darüber hinaus ist sie flexibel und anpassbar für verschiedene Aufgaben und Anwendungsfälle.


In den folgenden Abschnitten wird aufgezeigt, wie diese Methode umgesetzt wird und welche Vorteile sie bietet. Wir werden uns auf die Implementierung des Generators und Critic-Modells konzentrieren, sowie auf die Ergebnisse der Feedback-Schleife und die daraus gewonnenen Erkenntnisse.

### Aufgabe

Zeichne den Punkt P(4|6) in ein Koordinatensystem ein.

### Soll Lösung

1. Zeichne ein Koordinatensystem mit einer x-Achse und einer y-Achse, damit beide der Punkt im Koordinatensystem liegt <br>
		Achte darauf, die Achsen zu beschriften und eine regelmäßige Skalierung zu verwenden. <br>
2. Markiere den Ursprung (0, 0) des Koordinatensystems deutlich.<br>
3. Zeichne einen Vektor in x-Richtung, der von (0, 0) bis zum X Punkt reicht (x,  0). <br>
		Nutze eine klare Farbe wie Rot, um diesen Vektor zu kennzeichnen. <br>
4. Zeichne einen Vektor in y-Richtung, der von (0, 0) bis zum Y Punkt reicht (0, y). <br>
		Verwende eine andere Farbe, z. B. Grün. <br>
5. Verschiebe den y-Richtungs-Vektor so, dass sein Startpunkt das Ende des x-Richtungs-Vektors bei (x, 0) ist. <br>
6. Der Endpunkt dieses verschobenen Vektors markiert die Koordinaten (x, y).  <br>
		Zeichne einen Punkt an dieser Stelle und beschrifte ihn mit 'P({x-Wert}|{y-Wert})'. <br>
7. Füge ggf. Beschriftungen oder zusätzliche Hinweise hinzu, um die Schritte der Konstruktion verständlicher zu machen. <br>

In [ ]:
from openai import OpenAI

# Verbindung zum Ollama-Server herstellen
client = OpenAI(
    base_url='http://localhost:11434/v1',
    api_key='ollama',
)

In [ ]:
# Generator-Modell
def generate_visual_steps(task_description):
    response = client.chat.completions.create(
        model="phi4",
        messages=[
            {"role": "system", "content": (
                "Du bist ein Sprachmodell, das Mathematikaufgaben in präzise, nummerierte Schritte zerlegt, "
                "die leicht visuell dargestellt werden können. Gib die Antwort als eine Schritt-für-Schritt-Liste zurück, "
                "beginnend mit '1.', '2.', usw. Jede Nummer sollte einen klar definierten Schritt enthalten."
            )},
            {"role": "user", "content": task_description},
        ]
    )
    return response.choices[0].message.content 

In [ ]:
# Bewertungsmodell
def evaluate_solution(task_description, generated_solution):
    response = client.chat.completions.create(
        model="phi4",
        messages=[
            {"role": "system", "content": (
                "Du bist ein Sprachmodell, das Lösungen für Mathematikaufgaben bewertet. "
                "Bewerte die Lösung nicht nur auf Korrektheit, sondern auch darauf, ob sie die folgenden Kriterien erfüllt:\n"
                "1. Die Lösung muss als nummerierte Schritt-für-Schritt-Anleitung vorliegen.\n"
                "2. Die Schritte müssen klar und präzise formuliert sein.\n"
                "3. Begriffe wie 'Koordinatensystem', 'x-Achse', 'y-Achse', 'Vektor' sollten verwendet werden, "
                "wenn sie für die Aufgabe relevant sind.\n"
                "Antworte mit einer kurzen Bewertung und sage klar, ob die Lösung akzeptiert wird oder nicht."
            )},
            {"role": "user", "content": f"Aufgabe:\n{task_description}\n\nLösung:\n{generated_solution}"},
        ]
    )
    return response.choices[0].message.content

In [ ]:
# Chain-of-Judge
def interactive_solution(task_description, max_iterations=2):
    iteration = 0
    solution = generate_visual_steps(task_description)
    final_solution = None  
    print(f"Iteration {iteration + 1} - Generierte Lösung:\n{solution}")
    
    while iteration < max_iterations:
        evaluation = evaluate_solution(task_description, solution)
        print(f"Iteration {iteration + 1} - Bewertung der Lösung:\n{evaluation}")
        
        if "akzeptiert" in evaluation.lower() and "nummeriert" in evaluation.lower():
            print("Lösung akzeptiert!")
            final_solution = solution
            break
        else:
            iteration += 1
            if iteration < max_iterations:
                print(f"Neue Iteration {iteration + 1}: Verbesserung der Lösung...")
                solution = generate_visual_steps(task_description)
                print(f"Iteration {iteration + 1} - Neue generierte Lösung:\n{solution}")
            else:
                print("Maximale Anzahl an Iterationen erreicht. Keine weiteren Verbesserungen möglich.")
    
    if final_solution is None:
        final_solution = solution
    return final_solution  

In [ ]:
# Beispielaufgabe
task_description = "Zeichne den Punkt P(4|6) in ein Koordinatensystem ein."

# Ablauf mit maximal 2 Iterationen starten
final_solution = interactive_solution(task_description)